In [0]:
%run "/Functions/001_ConnectADLStoSynapse_1.0"

/_tests/testPath2/un1/
rodovias
/_tests/testPath2/un2/
rodovias

/_tests/testPath2/un1/
rodovias
/_tests/testPath2/un2/
rodovias

In [0]:
adls_synapse = ADLStoSynapseJDBC()

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import isnan, when, count, col
import pyspark.sql.functions as f
from pyspark.sql.types import *
from datetime import datetime
from datetime import timedelta
from pyspark.sql.window import Window
import calendar

In [0]:
#variaveis
tabela = 'receita_trafego'
#rodovias = ['autoban','spvias','viaoeste','rodoanel','dutra','msvia','vialagos','viasul','rodonorte','viario']
rodovias = ['rodonorte']
unidade = 'autoban'
path = f'raw/{unidade}/suat/{tabela}'
date_col = 'DTRECEITA'
sum_col1 = 'VLCALCULADO'
sum_col2 = ''
sum_col3 = ''
groupBy_col1 = 'CDCONCESS'
groupBy_col2 = 'DTRECEITA'
groupBy_col3 = ''
start_date = '01/04/2021'
end_date = '30/04/2021'
container = 'rodovias'
distinct_cols = ['NOPISTA','NOPRACA']

In [0]:
#Carrega a tabela parquet para iniciar as validações com a origem
df = adlsSynapse.read_parquet(container,path,None,None)


In [0]:
#Carrega a tabela com todas as unidades

df = None
for r in rodovias:
    df_carga = adlsSynapse.read_parquet(container,f'raw/{r}/suat/{tabela}',None,None)
    df_carga = df_carga.withColumn('UNIDADE',lit(r))

    if df is None:
        df = df_carga
    else:
        df = df.union(df_carga)  

In [0]:
#Filtra o dataframe para poder executar as validacoes no periodo
df = df.filter(f.from_unixtime(col(date_col)/1000, 'yyyy-MM-dd').between(start_date,end_date))
df = df.withColumn('DATA_RECEITA_FORMAT',f.from_unixtime(col(date_col)/1000, 'yyyy-MM-dd'))
df = df.drop('partition_dt')

In [0]:
display(df.head(2))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3216093880261697> in <module> 
 ----> 1 display ( df . head ( 2 ) ) 

 NameError : name 'df' is not defined

In [0]:
df.select(col('UNIDADE')==r)

Out[60]: DataFrame[(UNIDADE = rodonorte): boolean]

In [0]:
for r in rodovias:
  df_desc = df.filter(col('UNIDADE')==r)
  print(r)
  display(df_desc.describe())

rodonorte

summary,DATA_RECEITA,IDPRESTACAO,CDPAIS,CDCONCESS,NOPRACA,DTRECEITA,CDCATEGORIA,TPRECEITA,QTCALCULADA,VLCALCULADO,QTAJUSTADA,VLAJUSTADO,CDCAT_EQUIVALE,NOEIXOS_ADICIONAL,NOPISTA,VLCALCULADO_NORMAL,VLCALCULADO_DIFERENCA,UNIDADE,DATA_RECEITA_FORMAT
count,1833437,1833437,1833437,1833437,1833437,1833437,1833437,1833437,1833437,1833437,1833437,1833437,348136,1833437,1833437,1572822,1572822,1833437,1833437
mean,1.5821998979045366E12,100139.9550069078,618.0,2.0,138.27868096913065,1.5822862908835085E12,5.61487904956647,1.0964952709037725,41.18009836171082,818.234571026986,0.0,0.0,6.0,6.103923941755293,14.201674232602484,1025.7887023451992,226.30988115628864,null,null
stddev,2.0940510931325497E10,34330.627345291556,0.0,0.0,20.475907962766186,2.094051073997953E10,3.4796996659352204,0.29526933662198995,138.70789187175717,2248.6681686438087,0.0,0.0,0.0,36.9088546322288,31.118869041575888,2770.3142311106826,704.9291175682514,null,null
min,1546300800000,100018,0618,00002,0113,1546387199000,01,1,0,0,0,0,06,0,000,0,0,rodonorte,2019-01-01
max,1620518400000,99980,0618,00002,0174,1620604799000,20,2,999,9999.2,0,0,06,993,201,9999.2,9992.5,rodonorte,2021-05-09


In [0]:
df.columns

Out[62]: ['DATA_RECEITA',
 'IDPRESTACAO',
 'CDPAIS',
 'CDCONCESS',
 'NOPRACA',
 'DTRECEITA',
 'CDCATEGORIA',
 'TPRECEITA',
 'QTCALCULADA',
 'VLCALCULADO',
 'QTAJUSTADA',
 'VLAJUSTADO',
 'CDCAT_EQUIVALE',
 'NOEIXOS_ADICIONAL',
 'NOPISTA',
 'VLCALCULADO_NORMAL',
 'VLCALCULADO_DIFERENCA',
 'UNIDADE',
 'DATA_RECEITA_FORMAT']

In [0]:
#Total of columns
for r in rodovias:
  df_col = df.filter(col('UNIDADE')==r)
  print(r+' '+str(len(df_col.columns)))

rodonorte 19

In [0]:
#Find null in cols
for r in rodovias:
  df_null = df.filter(col('UNIDADE')==r).drop('partition_dt')
  print(r)
  display(df_null.select([count(when(isnan(c), c)).alias(c) for c in df_null.columns]))#.show()

rodonorte

DATA_RECEITA,IDPRESTACAO,CDPAIS,CDCONCESS,NOPRACA,DTRECEITA,CDCATEGORIA,TPRECEITA,QTCALCULADA,VLCALCULADO,QTAJUSTADA,VLAJUSTADO,CDCAT_EQUIVALE,NOEIXOS_ADICIONAL,NOPISTA,VLCALCULADO_NORMAL,VLCALCULADO_DIFERENCA,UNIDADE,DATA_RECEITA_FORMAT
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#distinct values

# for r in rodovias:
#   df_distinct = df.filter(col('UNIDADE')==r)
#   print(r)
#   for col in distinct_cols:
#     df.select(col).distinct().sort(col).show(df.count(),False)

In [0]:
#Values count
for r in rodovias:
  df_count = df.filter(col('UNIDADE')==r)
  print(r+' '+str(df_count.count()))

rodonorte 1833437

In [0]:
# Recria a data do group by para dias e mês 
df = df.withColumn(groupBy_col2+'_FORMAT', f.from_unixtime(col(groupBy_col2)/1000, 'yyyy-MM-dd'))
df = df.withColumn(groupBy_col2+'_FORMAT_MES', f.from_unixtime(col(groupBy_col2)/1000, 'yyyy-MM'))

In [0]:
#Converte as colunas para o formato numerico necessario para poder fazer os calculos
df = df.withColumn(sum_col1+'_CAST', col(sum_col1).cast(DecimalType(10,2)))

In [0]:
# Soma a coluna necessaria por mes para bater com a origem
for r in rodovias:
  df_sum_mes = df.filter(col('UNIDADE')==r)
  print(r)
  df_sum_mes.groupBy(groupBy_col2+'_FORMAT_MES').sum(sum_col1+'_CAST').sort(groupBy_col2+'_FORMAT_MES').show()

rodonorte
+--------------------+---------------------+
DTRECEITA_FORMAT_MES|sum(VLCALCULADO_CAST)|
+--------------------+---------------------+
 2019-01| 69259409.70|
 2019-02| 58327475.20|
 2019-03| 63311221.30|
 2019-04| 56848737.20|
 2019-05| 43085330.40|
 2019-06| 43478051.10|
 2019-07| 50682004.40|
 2019-08| 48444464.80|
 2019-09| 44958170.00|
 2019-10| 48241482.30|
 2019-11| 44877527.40|
 2019-12| 47531040.70|
 2020-01| 52014739.10|
 2020-02| 50052182.20|
 2020-03| 50195975.10|
 2020-04| 41906589.90|
 2020-05| 48015656.00|
 2020-06| 45717760.60|
 2020-07| 49699992.10|
 2020-08| 50040157.10|
+--------------------+---------------------+
only showing top 20 rows

In [0]:
# Soma a coluna por dia para bater com a origem
for r in rodovias:
  df_sum_dia = df.filter(col('UNIDADE')==r)
  print(r)
  df_sum_dia.groupBy(groupBy_col2+'_FORMAT').sum(sum_col1+'_CAST').sort(groupBy_col2+'_FORMAT').show()

rodonorte
+----------------+---------------------+
DTRECEITA_FORMAT|sum(VLCALCULADO_CAST)|
+----------------+---------------------+
 2019-01-01| 1368297.00|
 2019-01-02| 2684983.80|
 2019-01-03| 2625963.90|
 2019-01-04| 2391806.80|
 2019-01-05| 2142544.50|
 2019-01-06| 1955447.20|
 2019-01-07| 2016722.00|
 2019-01-08| 2260946.30|
 2019-01-09| 2325241.90|
 2019-01-10| 2388439.00|
 2019-01-11| 2403834.00|
 2019-01-12| 2058548.60|
 2019-01-13| 1976061.70|
 2019-01-14| 2145325.00|
 2019-01-15| 2422731.00|
 2019-01-16| 2453588.50|
 2019-01-17| 2537302.00|
 2019-01-18| 2461126.50|
 2019-01-19| 2012265.00|
 2019-01-20| 1928074.10|
+----------------+---------------------+
only showing top 20 rows

In [0]:
#Cria temp view para executar outros sql, caso necessário
df.createOrReplaceTempView('df')

In [0]:
#Count total de linhas
%sql select count(1)
      from 
      where   between '' and ''

File "<command-3216093880261681>" , line 3 
 from 
 ^
 IndentationError : unexpected indent

In [0]:
#Count por periodo (anual, mensal e diario)
%sql select    , count(1) 
     from
     where    between '' and ''
     group by 

File "<command-3216093880261682>" , line 3 
 from 
 ^
 IndentationError : unexpected indent